Install Required Packages and Libraries

In [23]:
!pip install llama-index llama-parse llama-index-embeddings-huggingface pyMuPDF

Lets download the CocoCola 10K-Filings Documents(WebScraping) from here https://investors.coca-colacompany.com/filings-reports/annual-filings-10-k

In [24]:
import requests
from bs4 import BeautifulSoup
import os

# URL of the Coca-Cola filings page
url = "https://investors.coca-colacompany.com/filings-reports/annual-filings-10-k"

# Directory to save the downloaded files
download_dir = "10k_filings"
os.makedirs(download_dir, exist_ok=True)

# Fetch the page content
response = requests.get(url)
response.raise_for_status()  # Raise an error if the request was unsuccessful

# Parse the page content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links to 10-K filings
filing_links = soup.find_all('a', href=True, title="PDF")
ten_k_links = [link['href'] for link in filing_links if '10-k' in link['href']]
# Download each 10-K filing
for link in ten_k_links:
    file_url = link if link.startswith('http') else 'https://investors.coca-colacompany.com' + link
    if file_url == url:
      continue
    file_name = os.path.join(download_dir, file_url.split('/')[-1])
    file_response = requests.get(file_url)
    with open(file_name, 'wb') as file:
        file.write(file_response.content)
    print(f"Downloaded {file_name}")

print("All 10-K filings downloaded.")


Downloaded 10k_filings/0001104659-24-035312.pdf
Downloaded 10k_filings/0000021344-24-000009.pdf
Downloaded 10k_filings/0000021344-23-000011.pdf
Downloaded 10k_filings/0000021344-22-000009.pdf
Downloaded 10k_filings/0000021344-21-000008.pdf
Downloaded 10k_filings/0000021344-20-000006.pdf
Downloaded 10k_filings/0000021344-19-000014.pdf
Downloaded 10k_filings/0000021344-18-000008.pdf
Downloaded 10k_filings/0000021344-17-000009.pdf
Downloaded 10k_filings/0000021344-16-000050.pdf
All 10-K filings downloaded.


In [25]:
# Read PDF using llama-parse
from llama_parse import LlamaParse
import nest_asyncio
import os
from google.colab import userdata

os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD') #LLAMA CLOUD API
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_1') #OPENAI_API_KEY_1
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
nest_asyncio.apply()

In [26]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader('/content/10k_filings/').load_data()[:3]
documents

[Document(id_='bb5822fa-7781-42ca-afbc-2f5c62b3b2ea', embedding=None, metadata={'page_label': '1', 'file_name': '0000021344-16-000050.pdf', 'file_path': '/content/10k_filings/0000021344-16-000050.pdf', 'file_type': 'application/pdf', 'file_size': 1744160, 'creation_date': '2024-08-09', 'last_modified_date': '2024-08-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' \nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\ný\n \nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended \nDecember 31, 2015\nOR\no\n \nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from \nto\nCommission File No. 001-022

In [27]:
# # bring in deps
# from llama_index.core import SimpleDirectoryReader

# # set up parser
# parser = LlamaParse(
#     result_type="markdown"  # "markdown" and "text" are available
# )

# # use SimpleDirectoryReader to parse our file
# file_extractor = {".pdf": parser}
# documents = SimpleDirectoryReader('/content/10k_filings/', file_extractor=file_extractor).load_data()#Can parse only 1000 pages per day(Free API key)
# print(documents)

# Alternative Approach using PyMuPDF(Rate Limit issue with LLAMAPARSER

In [28]:
# import fitz  # PyMuPDF
# import os

# # Directory containing the PDF files
# download_dir = "10k_filings"

# # Function to extract text from a PDF file
# def extract_text_from_pdf(pdf_path):
#     doc = fitz.open(pdf_path)
#     text = ""
#     for page in doc:
#         text += page.get_text()
#     return text

# # Read and extract text from all PDF files in the directory
# documents = []
# for pdf_file in os.listdir(download_dir):
#     if pdf_file.endswith('.pdf'):
#         pdf_path = os.path.join(download_dir, pdf_file)
#         print(f"Extracting text from {pdf_file} ...")
#         text = extract_text_from_pdf(pdf_path)
#         documents.append({'file_name': pdf_file, 'text': text})

# print(documents)


In [29]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text if hasattr(doc, "text") else doc['text'] for doc in documents]))
document

Document(id_='d9ce85cf-2f4a-4d9e-9699-0f4d1e6936f9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text=' \nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\ný\n \nANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended \nDecember 31, 2015\nOR\no\n \nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from \nto\nCommission File No. 001-02217\n(Exact name of Registrant as specified in its charter)\nDELAWARE\n(State or other jurisdiction of incorporation or organization)\n \n58-0628465\n(IRS Employer Identification No.)\nOne Coca-Cola Plaza\nAtlanta, Georgia\n(Address of principal executive offices)\n \n30313\n(Zip Code)\nRegistrant\'s telephone number, including area code: (404) 676-2121\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class\n

In [30]:
from llama_index.core.node_parser import SentenceWindowNodeParser
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=1,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
from llama_index.core import VectorStoreIndex
embed_model = "local:BAAI/bge-small-en-v1.5"
sentence_index = VectorStoreIndex.from_documents(
    [document], embed_model=embed_model, llm=llm)

In [31]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

In [32]:
sentwindow_query_engine = sentence_index.as_query_engine(
    similarity_top_k=2, node_postprocessors=[postproc]
)
response = sentwindow_query_engine.query('What is this about?')
print(response)

This is about a report from The Coca-Cola Company, which includes information on their business operations, financial data, risk factors, and forward-looking statements. It provides details on the company's market value, number of shares outstanding, executive officers, corporate governance, and other relevant information for investors and shareholders.


In [33]:
response = sentwindow_query_engine.query('What is the summary?')
print(response)

The Coca-Cola Company is a global beverage company that owns or licenses over 500 nonalcoholic beverage brands, including popular ones like Coca-Cola, Diet Coke, Fanta, and Sprite. They operate in more than 200 countries and have a vast distribution system. The company aims to provide consumers with a variety of beverage options while focusing on growth and creating value for shareholders.


In [34]:
from llama_index.core.llama_pack import download_llama_pack

RagEvaluatorPack = download_llama_pack(
  "RagEvaluatorPack", "./rag_evaluator_pack"
)

In [35]:
from llama_index.core import ServiceContext
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

gpt_4_llm = OpenAI(model="gpt-4", temperature=0.1)
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt_4_llm)

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    service_context=service_context_gpt4,
    num_questions_per_chunk=2,
    show_progress=True,
)

eval_dataset = dataset_generator.generate_dataset_from_nodes()
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=query_engine,
    judge_llm=gpt_4_llm,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)

<ipython-input-35-3f9cd19798b9>:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gpt4 = ServiceContext.from_defaults(llm=gpt_4_llm)


Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Batch processing of predictions: 100%|██████████| 8/8 [00:34<00:00,  4.30s/it]
Batch processing of evaluations:  73%|███████▎  | 4/5.5 [00:23<00:08,  5.79s/it]


In [36]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.687500
mean_relevancy_score,0.875000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.910724


# Automerging Retriever

In [37]:
from llama_index.core.node_parser import HierarchicalNodeParser
node_parser = HierarchicalNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(documents)
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
leaf_nodes = get_leaf_nodes(nodes)

from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

base_index = VectorStoreIndex(
    leaf_nodes,
    embed_model = embed_model,
    llm = llm,
    storage_context=storage_context,
)
from llama_index.core.retrievers.auto_merging_retriever import AutoMergingRetriever
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)
query_str = "What is the summary?"

nodes = retriever.retrieve(query_str)
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(retriever)
response = query_engine.query(query_str)
print(str(response))
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=query_engine,
    judge_llm=gpt_4_llm,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)


> Merging 4 nodes into parent node.
> Parent node id: 19ed6207-f88e-4c23-87ec-ab3ca615a6ef.
> Parent node text: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
...

> Merging 1 nodes into parent node.
> Parent node id: 21c4bfed-a1fe-49d9-bc56-a4cb148336f4.
> Parent node text: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
...

> Merging 4 nodes into parent node.
> Parent node id: 19ed6207-f88e-4c23-87ec-ab3ca615a6ef.
> Parent node text: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
...

> Merging 1 nodes into parent node.
> Parent node id: 21c4bfed-a1fe-49d9-bc56-a4cb148336f4.
> Parent node text: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
...

The Coca-Cola Company is the world's largest beverage company. The report covers various sections including business overview, risk factors, pro

Batch processing of predictions:   0%|          | 0/8 [00:00<?, ?it/s]

> Merging 4 nodes into parent node.
> Parent node id: 19ed6207-f88e-4c23-87ec-ab3ca615a6ef.
> Parent node text: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
...

> Merging 1 nodes into parent node.
> Parent node id: 21c4bfed-a1fe-49d9-bc56-a4cb148336f4.
> Parent node text: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
...

> Merging 2 nodes into parent node.
> Parent node id: a6717334-7414-4a7b-95db-0900eadab378.
> Parent node text: Yes 
o
    No 
ý
The aggregate market value of the common equity held by non-affiliates of the Re...

> Merging 2 nodes into parent node.
> Parent node id: a6717334-7414-4a7b-95db-0900eadab378.
> Parent node text: Yes 
o
    No 
ý
The aggregate market value of the common equity held by non-affiliates of the Re...

> Merging 4 nodes into parent node.
> Parent node id: 19ed6207-f88e-4c23-87ec-ab3ca615a6ef.
> Parent node text: Table of Contents
 
 
Page
 
For

Batch processing of predictions: 100%|██████████| 8/8 [00:08<00:00,  1.03s/it]
Batch processing of evaluations:  73%|███████▎  | 4/5.5 [00:30<00:11,  7.70s/it]


In [38]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.687500
mean_relevancy_score,0.875000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.943752


# Recursive Retriever


In [39]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core import Settings
doc_nodes = Settings.node_parser.get_nodes_from_documents(documents)
base_index = VectorStoreIndex(
    doc_nodes,
    embed_model = embed_model,
    llm = llm,
    storage_context=storage_context)
vec_retrv = base_index.as_retriever(top_k=1)
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vec_retrv},
    verbose=True
)
recretrv_query_engine = RetrieverQueryEngine.from_args(recursive_retriever)
response = recretrv_query_engine.query(query_str)
print(str(response))
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=recretrv_query_engine,
    judge_llm=gpt_4_llm,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)


Retrieving with query id None: What is the summary?
Retrieving text node: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
Risk Factors
11
Item 1B.
Unresolved Staff Comments
21
Item 2.
Properties
22
Item 3.
Legal Proceedings
22
Item 4.
Mine Safety Disclosures
24
Item X.
Executive Officers of the Company
24
Part II
 
 
Item 5.
Market for Registrant's Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
28
Item 6.
Selected Financial Data
31
Item 7.
Management's Discussion and Analysis of Financial Condition and Results of Operations
31
Item 7A.
Quantitative and Qualitative Disclosures About Market Risk
72
Item 8.
Financial Statements and Supplementary Data
74
Item 9.
Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
148
Item 9A.
Controls and Procedures
148
Item 9B.
Other Information
148
Part III
 
 
Item 10.
Directors, Executive Officers and Corporate Governance
148
Item 11.

Batch processing of predictions:   0%|          | 0/8 [00:00<?, ?it/s]

Retrieving with query id None: As of February 22, 2016, how many shares of the Registrant's Common Stock were outstanding?
Retrieving text node: Yes 
o
    No 
ý
The aggregate market value of the common equity held by non-affiliates of the Registrant (assuming for these purposes, but without conceding, that all executive officers and Directors are "affiliates" of the
Registrant) as of 
July 3, 2015
, the last business day of the Registrant's most recently completed second fiscal quarter, was $170,318,198,405 (based on the closing sale price of the Registrant's Common Stock
on that date as reported on the New York Stock Exchange).
The number of shares outstanding of the Registrant's Common Stock as of February 22, 2016, was 4,329,497,778.
DOCUMENTS INCORPORATED BY REFERENCE
Portions of the Company's Proxy Statement for the Annual Meeting of Shareowners to be held on 
April 27, 2016
, are incorporated by reference in Part III.
Retrieving text node: UNITED STATES
SECURITIES AND EXCHANGE C

Batch processing of predictions: 100%|██████████| 8/8 [00:05<00:00,  1.55it/s]
Batch processing of evaluations:  73%|███████▎  | 4/5.5 [01:21<00:30, 20.41s/it]


In [40]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.750000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.943966


As we can see, SentenceWindowRetriever and RecursiveRetriever are performing top-notch, former being slightly better between the two.

# SentenceWindowRetriever with SentenceTransformerRerank

In [41]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)
sent_query_engine = sentence_index.as_query_engine(
    similarity_top_k=2, node_postprocessors=[postproc, rerank]
)
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=sent_query_engine,
    judge_llm=gpt_4_llm,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)

Batch processing of predictions: 100%|██████████| 8/8 [01:33<00:00, 11.70s/it]
Batch processing of evaluations:  73%|███████▎  | 4/5.5 [00:23<00:08,  5.82s/it]


In [42]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.750000
mean_relevancy_score,0.750000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.910724


# Recursive Retriever with SentenceWindowTransformer RerAnk


In [45]:

recretrv_query_engine = RetrieverQueryEngine.from_args(recursive_retriever, node_postprocessors=[rerank])
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=recretrv_query_engine,
    judge_llm=gpt_4_llm,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)

Batch processing of predictions:   0%|          | 0/8 [00:00<?, ?it/s]

Retrieving with query id None: "In the context of a company's 10K filing, what information would you expect to find under 'Item 1A. Risk Factors' and 'Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations'? Provide a brief description of each."
Retrieving text node: Table of Contents
 
 
Page
 
Forward-Looking Statements
1
Part I
 
 
Item 1.
Business
1
Item 1A.
Risk Factors
11
Item 1B.
Unresolved Staff Comments
21
Item 2.
Properties
22
Item 3.
Legal Proceedings
22
Item 4.
Mine Safety Disclosures
24
Item X.
Executive Officers of the Company
24
Part II
 
 
Item 5.
Market for Registrant's Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
28
Item 6.
Selected Financial Data
31
Item 7.
Management's Discussion and Analysis of Financial Condition and Results of Operations
31
Item 7A.
Quantitative and Qualitative Disclosures About Market Risk
72
Item 8.
Financial Statements and Supplementary Data
74
Item 9.
Changes in an

Batch processing of predictions: 100%|██████████| 8/8 [00:36<00:00,  4.55s/it]
Batch processing of evaluations:  73%|███████▎  | 4/5.5 [00:25<00:09,  6.45s/it]


In [46]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.875000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.945206


# Router Query Engine - SentenceWindow + RecursiveRetriever

In [48]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors.pydantic_selectors import PydanticMultiSelector, PydanticSingleSelector

from llama_index.core.tools.query_engine import QueryEngineTool

# SentenceWindow Index tool
sent_tool = QueryEngineTool.from_defaults(
    query_engine=sent_query_engine,
    description="Useful when the context for a sentence is just below or above the sentences",
)

# Recursive Retriever Index tool
rec_tool = QueryEngineTool.from_defaults(
    query_engine=recretrv_query_engine,
    description="Useful for retrieving specific context from heirarichal contextual data.",
)

# Create Router Query Engine
ro_query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        sent_tool,
        rec_tool
    ],
)

rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=ro_query_engine,
    judge_llm=gpt_4_llm,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)

Batch processing of predictions:   0%|          | 0/8 [00:00<?, ?it/s]

Retrieving with query id None: According to the document, what are the different types of securities registered by the Registrant on the New York Stock Exchange?
Retrieving text node: Yes 
o
    No 
ý
The aggregate market value of the common equity held by non-affiliates of the Registrant (assuming for these purposes, but without conceding, that all executive officers and Directors are "affiliates" of the
Registrant) as of 
July 3, 2015
, the last business day of the Registrant's most recently completed second fiscal quarter, was $170,318,198,405 (based on the closing sale price of the Registrant's Common Stock
on that date as reported on the New York Stock Exchange).
The number of shares outstanding of the Registrant's Common Stock as of February 22, 2016, was 4,329,497,778.
DOCUMENTS INCORPORATED BY REFERENCE
Portions of the Company's Proxy Statement for the Annual Meeting of Shareowners to be held on 
April 27, 2016
, are incorporated by reference in Part III.
Retrieving text node: 

Batch processing of predictions: 100%|██████████| 8/8 [00:35<00:00,  4.44s/it]
Batch processing of evaluations:  73%|███████▎  | 4/5.5 [00:22<00:08,  5.64s/it]


In [49]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.625000
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.945193


In [51]:
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=sent_query_engine,
        metadata=ToolMetadata(name='sent', description="Useful when the context for a sentence is just below or above the sentences")
    ),
    QueryEngineTool(
        query_engine=recretrv_query_engine,
        metadata=ToolMetadata(name='recretrv', description="Useful for retrieving specific context from heirarichal contextual data.")
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)
rag_evaluator_pack = RagEvaluatorPack(
    rag_dataset=eval_dataset,
    query_engine=s_engine,
    judge_llm=gpt_4_llm,
)

base_benchmark = await rag_evaluator_pack.arun(
    batch_size=10,  # batches the number of openai api calls to make
    sleep_time_in_seconds=1,  # seconds to sleep before making an api call
)

Batch processing of predictions:   0%|          | 0/8 [00:00<?, ?it/s]

Generated 2 sub questions.
[sent] Q: What are forward-looking statements in the context of the report?
[sent] Q: Discuss the cautionary measures related to forward-looking statements in the report.
Generated 2 sub questions.
Generated 2 sub questions.
Generated 2 sub questions.
Generated 2 sub questions.
Generated 2 sub questions.
Generated 3 sub questions.
Generated 2 sub questions.
[sent] Q: What are the key factors that The Coca-Cola Company believes contribute to their success?
[recretrv] Q: Retrieve specific context from hierarchical data related to The Coca-Cola Company's success factors.
Retrieving with query id None: Retrieve specific context from hierarchical data related to The Coca-Cola Company's success factors.
Retrieving text node: FORWARD-LOOKING STATEMENTS
This report contains information that may constitute "forward-looking statements." Generally, the words "believe," "expect," "intend," "estimate," "anticipate," "project," "will"
and similar expressions identify forwa

Batch processing of predictions:  12%|█▎        | 1/8 [01:06<07:44, 66.38s/it]

[recretrv] A: Common Stock, Floating Rate Notes Due 2017, Floating Rate Notes Due 2019, 1.125% Notes Due 2022, 0.75% Notes Due 2023, 1.875% Notes Due 2026, 1.125% Notes Due 2027, and 1.625% Notes Due 2035 are the types of securities registered by the Registrant on the New York Stock Exchange.
[sent] A: Forward-looking statements in the context of the report are statements that address operating performance, events, or developments that are expected or anticipated to occur in the future. These statements include projections about volume growth, share of sales, earnings per share growth, and general views about future operating results.
[recretrv] A: The Coca-Cola Company's success factors include connecting with consumers by providing a wide variety of beverage options to meet their desires, needs, and lifestyles. Additionally, the company's success relies on the effective execution by its people every day. The company aims to leverage its assets such as brands, financial strength, dist

Batch processing of predictions:  25%|██▌       | 2/8 [01:06<02:45, 27.53s/it]

[sent] A: Caution should be taken not to place undue reliance on any forward-looking statements in the report because such statements speak only as of the date when made. The company undertakes no obligation to publicly update or revise any forward-looking statements, except as required by law. Additionally, forward-looking statements are subject to certain risks and uncertainties that could cause actual results to differ materially from historical experience and present expectations or projections.
[sent] A: The different types of securities registered by the Registrant on the New York Stock Exchange are Common Stock, Floating Rate Notes Due 2017, Floating Rate Notes Due 2019, 1.125% Notes Due 2022, 0.75% Notes Due 2023, 1.875% Notes Due 2026, 1.125% Notes Due 2027, 1.625% Notes Due 2035.
[sent] A: Item 10. Directors, Executive Officers and Corporate Governance in a company's 10K filing provides information about the individuals responsible for overseeing the company's operations, inc

Batch processing of predictions: 100%|██████████| 8/8 [01:08<00:00,  8.60s/it]
Batch processing of evaluations:  73%|███████▎  | 4/5.5 [02:30<00:56, 37.57s/it]


In [52]:
base_benchmark

rag,base_rag
metrics,
mean_correctness_score,4.812500
mean_relevancy_score,1.000000
mean_faithfulness_score,1.000000
mean_context_similarity_score,0.895529
